<a id='top'></a><a name='top'></a>
# Chapter 13: Advanced indexing with Annoy



* [Introduction](#introduction)
* [13.0 Imports and Setup](#13.0)
* [13.2 Optimizing NLP algorithms](#13.2)
    - [13.2.1 Indexing](#13.2.1)
    - [13.2.2 Advanced Indexing](#13.2.2)
    - [13.2.3 Advanced indexing with Annoy](#13.2.3)
    - [13.2.4 Why use approximate indexes at all?](#13.2.4)
    - [13.2.5 An indexing workaround: discretizing](#13.2.5)

---
<a name='introduction'></a><a id='introduction'></a>
# Introduction
<a href="#top">[back to top]</a>

### Datasets

* GoogleNews-vectors-negative300.bin.gz: [script](#GoogleNews-vectors-negative300.bin.gz), [source](https://www.dropbox.com/s/965dir4dje0hfi4/GoogleNews-vectors-negative300.bin.gz)


---
<a name='13.0'></a><a id='13.0'></a>
# 13.0 Imports and Setup
<a href="#top">[back to top]</a>

In [1]:
import os
if not os.path.exists('setup'):
    os.mkdir('setup')

In [2]:
req_file = "setup/requirements_13.txt"

In [3]:
%%writefile {req_file}
isort
scikit-learn-intelex
scrapy
watermark

Overwriting setup/requirements_13.txt


In [4]:
import sys
IS_COLAB = 'google.colab' in sys.modules

if IS_COLAB:
    print("Installing packages")
    !pip install --upgrade --quiet -r {req_file}
else:
    print("Running locally.")

Running locally.


In [5]:
#if IS_COLAB:
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [54]:
%%writefile setup/chp13_imports.py
import locale
import os
import os.path
import pprint
import random
import sys
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
from annoy import AnnoyIndex
from tqdm.auto import tqdm
from watermark import watermark
from gensim.models import KeyedVectors

Overwriting setup/chp13_imports.py


In [55]:
!isort setup/chp13_imports.py --sl
!cat setup/chp13_imports.py

Fixing /Users/gb/Desktop/examples/setup/chp13_imports.py
import locale
import os
import os.path
import pprint
import random
import sys
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
from annoy import AnnoyIndex
from gensim.models import KeyedVectors
from tqdm.auto import tqdm
from watermark import watermark


In [56]:
import locale
import os
import os.path
import pprint
import random
import sys
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
from annoy import AnnoyIndex
from gensim.models import KeyedVectors
from tqdm.auto import tqdm
from watermark import watermark

In [9]:
def HR():
    print("-"*40)
    
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"

locale.getpreferredencoding = getpreferredencoding
warnings.filterwarnings('ignore')
sns.set_style("darkgrid")
tqdm.pandas(desc="progress-bar")
pp = pprint.PrettyPrinter(indent=4)
random.seed(42)
np.random.seed(42)

print(watermark(iversions=True,globals_=globals(),python=True,machine=True))

Python implementation: CPython
Python version       : 3.8.12
IPython version      : 7.34.0

Compiler    : Clang 13.0.0 (clang-1300.0.29.3)
OS          : Darwin
Release     : 21.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

seaborn: 0.12.1
numpy  : 1.23.5
sys    : 3.8.12 (default, Dec 13 2021, 20:17:08) 
[Clang 13.0.0 (clang-1300.0.29.3)]



---
<a name='13.2'></a><a id='13.2'></a>
# 13.2 Optimizing NLP algorithms
<a href="#top">[back to top]</a>

<a name='13.2.1'></a><a id='13.2.1'></a>
## 13.2.1 Indexing
<a href="#top">[back to top]</a>

<a name='13.2.2'></a><a id='13.2.2'></a>
## 13.2.2 Advanced indexing
<a href="#top">[back to top]</a>

<a name='13.2.3'></a><a id='13.2.3'></a>
## 13.2.3 Advanced indexing with Annoy
<a href="#top">[back to top]</a>

<a id='GoogleNews-vectors-negative300.bin.gz'></a><a name='GoogleNews-vectors-negative300.bin.gz'></a>
### Dataset: GoogleNews-vectors-negative300.bin.gz
<a href="#top">[back to top]</a>

In [10]:
%%time

# Download word2vec vectors
data_gn_dir = 'data/data_word2vec'
if not os.path.exists(data_gn_dir):
    os.makedirs(data_gn_dir)

data_gn_file = 'GoogleNews-vectors-negative300.bin.gz'
data_gn_path = f"{data_gn_dir}/{data_gn_file}"
!wget -P {data_gn_dir} -O {data_gn_path} -nc "https://www.dropbox.com/s/965dir4dje0hfi4/GoogleNews-vectors-negative300.bin.gz/GoogleNews-vectors-negative300.bin.gz?dl=1"
!du -h {data_gn_path}

File ‘data/data_word2vec/GoogleNews-vectors-negative300.bin.gz’ already there; not retrieving.
1.5G	data/data_word2vec/GoogleNews-vectors-negative300.bin.gz
CPU times: user 15.4 ms, sys: 24.2 ms, total: 39.6 ms
Wall time: 516 ms


In [11]:
%%time

print("Loading word2vec model")
wv = KeyedVectors.load_word2vec_format(data_gn_path, binary=True)
print("Done loading model.")

Loading word2vec model
Done loading model.
CPU times: user 1min 8s, sys: 5.33 s, total: 1min 14s
Wall time: 1min 20s


In [12]:
# Get shape of word2vec vectors
num_words, num_dimensions = wv.vectors.shape
print(f"Rows: {num_words:,}")
print(f"Columns: {num_dimensions:,}")

Rows: 3,000,000
Columns: 300


### AnnoyIndex is a class that provides functions for k-nearest neighbors search.

`AnnoyIndex(f, metric)` returns a new index that's read-write and stores vector of f dimensions. Metric can be "angular", "euclidean", "manhattan", "hamming", or "dot".

Defaults to: 'angular'

### Initiate 300D AnnoyIndex

In [13]:
%%time

# Create a new index that is read-write and stores vector of num_dimensions.
index = AnnoyIndex(
    num_dimensions, 
    'euclidean'
)

index.set_seed(42)

CPU times: user 64 µs, sys: 27 µs, total: 91 µs
Wall time: 102 µs


### Add each word2vec vector to Annoy index

    wv.index_to_key: Built-in mutable sequence.

In [14]:
%%time

for i, word in enumerate(tqdm(wv.index_to_key)):
    # Visual check
    if not i % 100_000:
        print(f"{i:,}: {word}") 
    index.add_item(i, wv[word])

HR()
print("Done")
HR()

  0%|          | 0/3000000 [00:00<?, ?it/s]

0: </s>
100,000: distinctiveness
200,000: barbiturate
300,000: Sony_PS3
400,000: Infiniti_FX
500,000: Attorney_Bud_Cummins
600,000: Giske
700,000: f_***_er
800,000: Shaw_Stockbroking_Ltd.
900,000: HKSTP
1,000,000: Starwood_Hotels_HOT
1,100,000: McGrath_RentCorp_NASDAQ_MGRC
1,200,000: Piveteau
1,300,000: Rob_Pavey
1,400,000: Giant_Octopus
1,500,000: eur_UPM_Kymmene
1,600,000: CSSL
1,700,000: Lubina
1,800,000: Ndian
1,900,000: Cape_Solander
2,000,000: Iordanis
2,100,000: Allegiance_recitation
2,200,000: brandy_soaked
2,300,000: Coach_Kurt_Budke
2,400,000: backcountry_hikers
2,500,000: Brawn_BMW_Sauber
2,600,000: cedar_juniper
2,700,000: Wendy_Liberatore
2,800,000: Management_GDCM
2,900,000: BOARDED_UP
----------------------------------------
Done
----------------------------------------
CPU times: user 2min 16s, sys: 15.8 s, total: 2min 31s
Wall time: 2min 57s


### Build Euclidean distance index with 15 trees

Next, this `AnnoyIndex` object reads through the entire index and clusters the vectors into chunks that can be indexed into a tree structure.

In [15]:
%%time

annoy_euclidean_index_file = 'annoy_euclidean_index.ann'

# hyperparameter
num_trees = int(np.log(num_words).round(0))
print(f"{num_trees} trees for our {num_words:,} vectors.")   

print(f"Building Euclidean distance indexing for {num_trees} trees.")
index.build(num_trees)

# Build index on disk to enable indexing big datasets that won't fit into memory.
print(f"Saving Annoy index file as {annoy_euclidean_index_file}")
index.save(annoy_euclidean_index_file)

15 trees for our 3,000,000 vectors.
Building Euclidean distance indexing for 15 trees.
Saving Annoy index file as annoy_euclidean_index.ann
CPU times: user 5min 14s, sys: 49.5 s, total: 6min 3s
Wall time: 2min 23s


True

In [16]:
!du -h word2vec_index.ann

4.0G	word2vec_index.ann


In [17]:
%%time
w2id = dict(zip(range(len(wv.key_to_index)), wv.key_to_index))

CPU times: user 668 ms, sys: 2.67 s, total: 3.34 s
Wall time: 12.9 s


In [18]:
f"{len(w2id):,}"

'3,000,000'

In [19]:
for i, x in enumerate(w2id):
    if i >= 10:
        break
    print(i, x)

0 0
1 1
2 2
3 3
4 4
5 5
6 6
7 7
8 8
9 9


### Look up words from our vocabulary in the AnnoyIndex

In [20]:
# Find Harry_Potter neighbors with AnnoyIndex
wv.key_to_index['Harry_Potter']

9494

In [21]:
# Look up number of times the Harry_Potter 20gram was mentioned in the GoogleNews corpus.
wv.get_vecattr("Harry_Potter", "count")

2990506

In [22]:
# Create a map similar to wv.key_to_index mapping the tokens to their index values (integer)
w2id = dict(zip(wv.key_to_index, range(len(wv.key_to_index))))

In [23]:
w2id['Harry_Potter']

9494

In [24]:
ids = index.get_nns_by_item(
    w2id['Harry_Potter'], 11
)
ids

[9494,
 37681,
 597774,
 307186,
 987327,
 481759,
 500570,
 728042,
 400305,
 872473,
 501614]

### Top 10 nearest neighbors listed by Annoy. 

This provides results from the general vicinity of a search term, rather than the closest 10. 

In [37]:
results_euclidean = [wv.index_to_key[i] for i in ids]
results_euclidean

['Harry_Potter',
 'JK_Rowling',
 'Neville_Longbottom',
 'muggle',
 'Professor_Slughorn',
 'Harry_Potter_Daniel_Radcliffe',
 'Death_Eater',
 'Hogwarts_headmaster_Albus_Dumbledore',
 'Hogwart',
 'Remus_Lupin',
 'Lucius_Malfoy']

### Top Harry_Potter neighbors with gensim.KeyedVectors index

This looks more like a relevant top-10 synonym list. 

In [36]:
%%time
print("Start")
results_gensim = [word for word, similarity in wv.most_similar('Harry_Potter', topn=10)]
results_gensim

Start
CPU times: user 626 ms, sys: 43.5 ms, total: 669 ms
Wall time: 375 ms


['JK_Rowling_Harry_Potter',
 'JK_Rowling',
 'boy_wizard',
 'Deathly_Hallows',
 'Half_Blood_Prince',
 'Rowling',
 'Actor_Rupert_Grint',
 'HARRY_Potter',
 'wizard_Harry_Potter',
 'HARRY_POTTER']

### Rebuild the Annoy indexing approprimation to use the cosine distance metric.

* Rebuild with cosine instead of Euclidean, and add more trees. 
* This should improve the accuracy of the nearest neighbors and make its results match gensim's more closely.

In [27]:
%%time

print("Start")

index_cos = AnnoyIndex(
    f=num_dimensions, 
    metric='angular'
)

for i, word in enumerate(tqdm(wv.index_to_key)):
    # Visual check
    if not i % 100_000:
        print(f"{i:,}: {word}") 
    index_cos.add_item(i, wv[word])

HR()
print("Done")
HR()

Start


  0%|          | 0/3000000 [00:00<?, ?it/s]

0: </s>
100,000: distinctiveness
200,000: barbiturate
300,000: Sony_PS3
400,000: Infiniti_FX
500,000: Attorney_Bud_Cummins
600,000: Giske
700,000: f_***_er
800,000: Shaw_Stockbroking_Ltd.
900,000: HKSTP
1,000,000: Starwood_Hotels_HOT
1,100,000: McGrath_RentCorp_NASDAQ_MGRC
1,200,000: Piveteau
1,300,000: Rob_Pavey
1,400,000: Giant_Octopus
1,500,000: eur_UPM_Kymmene
1,600,000: CSSL
1,700,000: Lubina
1,800,000: Ndian
1,900,000: Cape_Solander
2,000,000: Iordanis
2,100,000: Allegiance_recitation
2,200,000: brandy_soaked
2,300,000: Coach_Kurt_Budke
2,400,000: backcountry_hikers
2,500,000: Brawn_BMW_Sauber
2,600,000: cedar_juniper
2,700,000: Wendy_Liberatore
2,800,000: Management_GDCM
2,900,000: BOARDED_UP
----------------------------------------
Done
----------------------------------------
CPU times: user 2min 25s, sys: 16.8 s, total: 2min 42s
Wall time: 3min 23s


### Build twice the number of trees, with cosine distance index

The indexing should take twice as long to run, but once it finished we can expect results closer to what gensim produces. 

In [28]:
%%time
print("Start")
annoy_cos_index_file = 'annoy_cos_index.ann'

index_cos.build(30)

Start
CPU times: user 15min 22s, sys: 1min 4s, total: 16min 27s
Wall time: 5min 51s


True

In [29]:
%%time
print("Start")
index_cos.save(annoy_cos_index_file)

Start
CPU times: user 775 ms, sys: 16.8 s, total: 17.6 s
Wall time: 33.3 s


True

In [30]:
!du -h {annoy_cos_index_file}

4.6G	annoy_cos_index.ann


### Harry_Potter neighbors in a cosine distance world

In [31]:
ids_cos = index_cos.get_nns_by_item(
    w2id['Harry_Potter'], 10
)

ids_cos

[9494, 193309, 40544, 41526, 340510, 1038917, 14273, 165465, 337152, 1889661]

In [34]:
results_cos = [wv.index_to_key[i] for i in ids_cos]
results_cos

['Harry_Potter',
 'JK_Rowling_Harry_Potter',
 'Deathly_Hallows',
 'Half_Blood_Prince',
 'wizard_Harry_Potter',
 'OotP',
 'Twilight',
 'Twilight_saga',
 'Stephenie_Meyer_Twilight',
 'Harry_Potter_Hermione_Granger']

### Comparison of Top-10 Search Results

In [51]:
df = pd.DataFrame(
    list(
        zip(
            results_gensim, 
            results_euclidean, 
            results_cos
        )
    ), 
    columns =['annoy_gensim','annoy_euclidean_15', 'annoy_cosine_30'],
)

df

,annoy_gensim,annoy_euclidean_15,annoy_cosine_30
0,JK_Rowling_Harry_Potter,Harry_Potter,Harry_Potter
1,JK_Rowling,JK_Rowling,JK_Rowling_Harry_Potter
2,boy_wizard,Neville_Longbottom,Deathly_Hallows
3,Deathly_Hallows,muggle,Half_Blood_Prince
4,Half_Blood_Prince,Professor_Slughorn,wizard_Harry_Potter
5,Rowling,Harry_Potter_Daniel_Radcliffe,OotP
6,Actor_Rupert_Grint,Death_Eater,Twilight
7,HARRY_Potter,Hogwarts_headmaster_Albus_Dumbledore,Twilight_saga
8,wizard_Harry_Potter,Hogwart,Stephenie_Meyer_Twilight
9,HARRY_POTTER,Remus_Lupin,Harry_Potter_Hermione_Granger


<a name='13.2.4'></a><a id='13.2.4'></a>
## 13.2.4 Why use approximate indexes at all?
<a href="#top">[back to top]</a>

<a name='13.2.5'></a><a id='13.2.5'></a>
## 13.2.5 An indexing workaround: discretizing
<a href="#top">[back to top]</a>